In [1]:
from sktime.classification.hybrid import HIVECOTEV2
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

In [2]:
spike_vector = np.load('spikes_vector_1000.npy')
plain_vector = np.load('plain_vector_1000.npy')

# Create labels for spikes (1) and plain data (0)
spike_labels = np.ones(spike_vector.shape[0])  # Label for spikes
plain_labels = np.zeros(plain_vector.shape[0])  # Label for plain data

# Combine data and labels
data = np.concatenate((spike_vector, plain_vector), axis=0)
labels = np.concatenate((spike_labels, plain_labels), axis=0)

# Shuffle the data and create train-test split
data_train, data_test, labels_train, labels_test = train_test_split(
    data, labels, test_size=0.2, random_state=42, stratify=labels
)

# Convert data to DataFrame if needed (HIVECOTEV2 can work with pandas DataFrame)
data_train_df = pd.DataFrame(data_train)
data_test_df = pd.DataFrame(data_test)

# Convert labels to pandas Series (or keep as numpy arrays)
labels_train_series = pd.Series(labels_train)
labels_test_series = pd.Series(labels_test)

In [3]:
classifier = HIVECOTEV2()
classifier.fit(data_train_df, labels_train_series)
y_pred = classifier.predict(data_test_df)

accuracy = accuracy_score(labels_test_series, y_pred)
print(f"Accuracy: {accuracy}")

TypeError: X must be in an sktime compatible format. Allowed scitypes for classifiers are Panel mtypes, for instance a pandas.DataFrame with MultiIndex and last(-1) level an sktime compatible time index. Allowed compatible mtype format specifications are: ['nested_univ', 'numpy3D', 'numpyflat', 'pd-multiindex', 'pd-wide', 'pd-long', 'df-list', 'gluonts_ListDataset_panel', 'gluonts_PandasDataset_panel', 'polars_panel'] . See the data format tutorial examples/AA_datatypes_and_datasets.ipynb. If you think the data is already in an sktime supported input format, run sktime.datatypes.check_raise(data, mtype) to diagnose the error, where mtype is the string of the type specification you want. Error message for checked mtypes, in format [mtype: message], as follows: [df-list: obj must be list of pd.DataFrame, found <class 'pandas.core.frame.DataFrame'>]  [nested_univ: obj All columns must be object, found <class 'pandas.core.frame.DataFrame'>]  [numpy3D: obj must be a numpy.ndarray, found <cl